<a href="https://colab.research.google.com/github/aswinaus/Agents/blob/main/Graph_RAG_Tax_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install pyvis IPython cchardet datasets langchain==0.1.17 neo4j openai tiktoken langchain-community langchain-experimental json-repair

In [ ]:
from getpass import getpass
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [ ]:
from datasets import load_dataset
import pandas as pd
dataset = load_dataset("aswinaus/Individual_income_tax_dataset_state_zipcode_2019", download_mode="force_redownload")
df=pd.DataFrame(dataset['train'])

README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

(…)statistics_dataset_per_state_zipcode.csv:   0%|          | 0.00/80.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/165890 [00:00<?, ? examples/s]

In [5]:
df.head(10)

,STATEFIPS,STATE,zipcode,Size of adjusted gross income,No of returns,No of single returns,No of joint returns,No of head of household returns,Number of electronically filed returns,Number of computer prepared paper returns,...,Number of returns with net investment income tax,Net investment income tax amount,Number of returns with tax due at time of filing,Tax due at time of filing amount,Number of returns with total overpayments,Total overpayments amount,Number of returns with overpayments refunded,Overpayments refunded amount,Number of returns with credit to next years estimated tax,Credited to next years estimated tax amount
0,1,AL,35004,"$1 under $25,000",1450,990,180,250,1320,60,...,0,0,120,100,1230,2383,1220,2365,0,0
1,1,AL,35004,"$25,000 under $50,000",1370,700,280,340,1260,50,...,0,0,220,261,1150,2995,1140,2984,0,0
2,1,AL,35004,"$50,000 under $75,000",970,410,370,160,900,50,...,0,0,240,410,730,1814,720,1799,40,46
3,1,AL,35004,"$75,000 under $100,000",650,130,460,60,610,0,...,0,0,170,391,480,1666,470,1468,0,0
4,1,AL,35004,"$100,000 under $200,000",800,80,690,30,740,40,...,0,0,320,992,490,1606,480,1535,30,274
5,1,AL,35004,"$200,000 or more",90,0,80,0,80,0,...,20,30,50,564,40,586,30,263,0,0
6,1,AL,35005,"$1 under $25,000",1330,800,130,380,1200,50,...,0,0,80,58,1190,3344,1190,3344,0,0
7,1,AL,35005,"$25,000 under $50,000",1010,470,190,330,940,50,...,0,0,140,138,870,2559,870,2556,0,0
8,1,AL,35005,"$50,000 under $75,000",510,190,180,140,460,0,...,0,0,120,243,390,1050,390,1046,0,0
9,1,AL,35005,"$75,000 under $100,000",230,40,140,50,200,20,...,0,0,60,139,170,635,170,635,0,0


In [ ]:
# @title Knowledge Graph Builder & Visualizer
# @markdown Note that rendering the graph will take a minute or two under the default 10% sample size, longer with higher sample sizes.
sample_size = 0.5 # @param {type:"number", default:0.10}
import pandas as pd
import networkx as nx
from pyvis.network import Network
from IPython.display import IFrame
from IPython.display import Markdown, HTML


colors = {
    'zipcode': 'lightblue',
    'incometaxby_state_name': 'orange',
    'No of returns': 'red',
    'No of single returns': 'green',
    'No of joint returns': 'magenta',
    'No of joint returns': 'purple',
}
sizes = {
    'zipcode': 20,
    'Size of adjusted gross income': 15,
    'No of returns': 25,
    'No of single returns': 30,
    'No of joint returns': 10
}

# Stratify based on 'rating_value', 'locality', 'trip_period', 'hotel_name' columns
stratify_cols = ['Size of adjusted gross income', 'zipcode', 'STATE', 'No of returns']
sample_size = 0.6

# Perform stratified sampling
sampled_df = df.groupby(stratify_cols, group_keys=False).apply(lambda x: x.sample(frac=sample_size))

# Initialize a directed graph
G = nx.DiGraph()

# Adding nodes with the entity type as a node attribute
for index, row in sampled_df.iterrows():
    incometaxby_state_name=f"{row['STATE']}_{row['zipcode']}_{row['Size of adjusted gross income']}"
    if row['STATE'] not in G:
      G.add_node(row['STATE'],
                 entity='STATE',
                 color=colors.get('STATE', 'blue'),
                 size=sizes.get('STATE', 5), )

    if row['Size of adjusted gross income'] not in G:
      G.add_node(row['Size of adjusted gross income'], entity='Size of adjusted gross income', color=colors.get('LOCALITY', 'gray'),
                 size=sizes.get('Size of adjusted gross income', 40))

    G.add_node(
               row['zipcode'],
               entity='ZIPCODE',
               color=colors.get('ZIPCODE', 'orange'),
               size=sizes.get('ZIPCODE', 20))

    G.add_node(
               row['No of returns'],
               entity='No_of_returns',
               color=colors.get('No_of_returns', 'green'),
               size=sizes.get('No_of_returns', 25))

    G.add_node(
               row['No of single returns'],
               entity='No_of_single_returns',
               color=colors.get('No_of_single_returns', 'orange'),
               size=sizes.get('No_of_returns', 25))

    G.add_node(
               row['No of joint returns'],
               entity='No_of_joint_returns',
               color=colors.get('No_of_joint_returns', 'brown'),
               size=sizes.get('No_of_joint_returns',25))

    G.add_edge(row['STATE'], row['zipcode'],relationship='Size of adjusted gross income')
    G.add_edge(row['zipcode'], row['No of single returns'],relationship='Size of adjusted gross income')
    G.add_edge(row['zipcode'], row['No of joint returns'],relationship='Size of adjusted gross income')

    G.add_edge(row['zipcode'],row['No of returns'], relationship='Size of adjusted gross income')


# Step 4: Visualization (Optional)
# Convert to a pyvis network
nt = Network('700px', '700px', notebook=True, cdn_resources='in_line')
# if you are not in a Jupyter environment, you might need to set notebook=False
nt.from_nx(G)
nt.toggle_physics(True)  # Enable force-directed algorithm
nt.save_graph('income_tax_2019_graph.html')
nt.show('income_tax_2019_graph.html')

HTML('income_tax_2019_graph.html')

<ipython-input-7-cb1c545fe58a>:32: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_df = df.groupby(stratify_cols, group_keys=False).apply(lambda x: x.sample(frac=sample_size))
